In [1]:
pip install pygame pydub



   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ----------------

In [2]:
import numpy as np
import pandas as pd

# Step 1: Load the .npy file
npy_file_path = "images.npy"  
data = np.load(npy_file_path)

# Step 2: Inspect the data
print("Shape of the loaded data:", data.shape)
print("First few rows of the data:\n", data[:5])

Shape of the loaded data: (17736, 256, 256, 3)
First few rows of the data:
 [[[[ 68  19  23]
   [ 58  17  33]
   [ 40  16  40]
   ...
   [ 51  46  43]
   [ 40  41  33]
   [ 28  38  30]]

  [[ 61  40  37]
   [ 52  43  44]
   [ 29  39  41]
   ...
   [ 28  30  42]
   [ 28  32  35]
   [ 33  40  33]]

  [[ 30  31  25]
   [ 24  41  33]
   [ 15  45  33]
   ...
   [ 21  35  48]
   [ 25  35  45]
   [ 37  41  40]]

  ...

  [[ 28  32  35]
   [ 27  31  32]
   [ 42  44  43]
   ...
   [ 20  37  27]
   [ 19  40  21]
   [ 27  50  24]]

  [[ 22  36  37]
   [ 19  29  30]
   [ 33  37  38]
   ...
   [ 17  35  35]
   [ 17  35  21]
   [ 26  45  17]]

  [[ 18  39  40]
   [ 14  28  29]
   [ 24  30  30]
   ...
   [ 20  35  56]
   [ 27  37  36]
   [ 28  36  15]]]


 [[[ 19  26  42]
   [ 15  27  39]
   [ 15  22  32]
   ...
   [ 32  37  33]
   [ 34  23  21]
   [ 66  35  40]]

  [[  9  29  20]
   [ 31  37  33]
   [ 31  27  26]
   ...
   [ 31  37  33]
   [ 34  31  26]
   [ 42  28  27]]

  [[  3  25   2]
   [ 38  4

In [5]:
# Step 3: Convert to DataFrame (if data represents RGB values)
if data.shape[1] == 3:  # Assuming 3 columns for R, G, B
    df = pd.DataFrame(data, columns=["R", "G", "B"])

In [7]:

# Step 1: Flatten the first few images into rows of RGB values
subset = data[:5]  # Take the first 5 images for the subset
flattened = subset.reshape(-1, 3)  # Reshape to (5 * 256 * 256, 3)

# Step 2: Create a DataFrame with RGB columns
df = pd.DataFrame(flattened, columns=["R", "G", "B"])

# Step 3: Save the DataFrame as a CSV file
csv_path = "galaxy_image_pixel_data.csv"
df.to_csv(csv_path, index=False)
print(f"CSV file saved as: {csv_path}")


CSV file saved as: galaxy_image_pixel_data.csv


In [8]:
import numpy as np
import pandas as pd
from pydub import AudioSegment
from pydub.generators import Sine


c:\Users\Asus\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


##### Mapping pixels to pitch

In [11]:

df = pd.read_csv('galaxy_image_pixel_data.csv') 

# Take a subset for demonstration (e.g., first 1000 rows)
df_subset = df.head(1000)

# Function to map brightness to pitch
def map_brightness_to_pitch(r, g, b):
    brightness = 0.299 * r + 0.587 * g + 0.114 * b  # Calculate brightness
    return 220 + (brightness / 255) * 660  # Map brightness to 220-880 Hz

# Generate audio from pixel data
def generate_audio_pixel_to_pitch(df_subset):
    audio = AudioSegment.silent(duration=0)

    for _, row in df_subset.iterrows():
        pitch = map_brightness_to_pitch(row['R'], row['G'], row['B'])
        sine_wave = Sine(pitch).to_audio_segment(duration=100)  # Fixed duration of 100 ms
        audio += sine_wave

    return audio


In [12]:
# Generate audio and export
output_path = ".\Mapped_sounds\pixel_to_pitch.wav"
audio = generate_audio_pixel_to_pitch(df_subset)
audio.export(output_path, format="wav")
print(f"WAV file saved at: {output_path}")


<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Asus\AppData\Local\Temp\ipykernel_21700\3407770205.py:2: SyntaxWarning: invalid escape sequence '\M'
  output_path = ".\Mapped_sounds\pixel_to_pitch.wav"


WAV file saved at: .\Mapped_sounds\pixel_to_pitch.wav


##### Mapping Pixels to Volume 

In [13]:

df = pd.read_csv('galaxy_image_pixel_data.csv') 

# Take a subset for demonstration (e.g., first 1000 rows)
df_subset = df.head(1000)

# Function to map red channel to volume
def map_red_to_volume(r):
    return (r / 255) * 20 - 20  # Map 0-255 to -20 to 0 dB

# Generate audio from pixel data
def generate_audio_pixel_to_volume(df_subset):
    audio = AudioSegment.silent(duration=0)

    for _, row in df_subset.iterrows():
        volume = map_red_to_volume(row['R'])
        sine_wave = Sine(440).to_audio_segment(duration=100).apply_gain(volume)  # Fixed pitch 440 Hz
        audio += sine_wave

    return audio


In [15]:
# Generate audio and export
output_path = ".\Mapped_sounds\pixel_to_volume.wav"
audio = generate_audio_pixel_to_volume(df_subset)
audio.export(output_path, format="wav")
print(f"WAV file saved at: {output_path}")


<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Asus\AppData\Local\Temp\ipykernel_21700\1645185491.py:2: SyntaxWarning: invalid escape sequence '\M'
  output_path = ".\Mapped_sounds\pixel_to_volume.wav"


WAV file saved at: .\Mapped_sounds\pixel_to_volume.wav


##### Mapping Pixels to Duration

In [16]:
# Function to map blue channel to duration
def map_blue_to_duration(b):
    return 100 + (b / 255) * 400  # Map 0-255 to 100-500 ms

# Generate audio from pixel data
def generate_audio_pixel_to_duration(df_subset):
    audio = AudioSegment.silent(duration=0)

    for _, row in df_subset.iterrows():
        duration = map_blue_to_duration(row['B'])
        sine_wave = Sine(440).to_audio_segment(duration=duration)  # Fixed pitch 440 Hz
        audio += sine_wave

    return audio



In [17]:
# Generate audio and export
output_path = ".\Mapped_sounds\pixel_to_duration.wav"
audio = generate_audio_pixel_to_duration(df_subset)
audio.export(output_path, format="wav")
print(f"WAV file saved at: {output_path}")

<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Asus\AppData\Local\Temp\ipykernel_21700\2239316087.py:2: SyntaxWarning: invalid escape sequence '\M'
  output_path = ".\Mapped_sounds\pixel_to_duration.wav"


WAV file saved at: .\Mapped_sounds\pixel_to_duration.wav
